In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [14]:
import numpy as np
xy = np.loadtxt('data-01-test-score.csv',delimiter=',',dtype=np.float32)
#단점 : 같은 데이터 타입이여야 한다.
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]

print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data)

(25, 3) [[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]
 [ 69.  74.  77.]
 [ 47.  56.  60.]
 [ 87.  79.  90.]
 [ 79.  70.  88.]
 [ 69.  70.  73.]
 [ 70.  65.  74.]
 [ 93.  95.  91.]
 [ 79.  80.  73.]
 [ 70.  73.  78.]
 [ 93.  89.  96.]
 [ 78.  75.  68.]
 [ 81.  90.  93.]
 [ 88.  92.  86.]
 [ 78.  83.  77.]
 [ 82.  86.  90.]
 [ 86.  82.  89.]
 [ 78.  83.  85.]
 [ 76.  83.  71.]
 [ 96.  93.  95.]] 25
(25, 1) [[152.]
 [185.]
 [180.]
 [196.]
 [142.]
 [101.]
 [149.]
 [115.]
 [175.]
 [164.]
 [141.]
 [141.]
 [184.]
 [152.]
 [148.]
 [192.]
 [147.]
 [183.]
 [177.]
 [159.]
 [177.]
 [175.]
 [175.]
 [149.]
 [192.]]


In [15]:
X = tf.placeholder(tf.float32, shape=[None,3])
Y = tf.placeholder(tf.float32, shape=[None,1])

In [16]:
W = tf.Variable(tf.random_normal([3,1]),name='weight')
b = tf.Variable(tf.random_normal([1]),name='bias')

In [17]:
hypothesis = tf.matmul(X,W)+b

In [18]:
cost = tf.reduce_mean(tf.square(hypothesis-Y))
#reduce_mean은 특정 차원을 제거하고 평균을 구한다.

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(2001):
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 [184.55312]
 [169.92015]
 [176.35362]
 [152.5017 ]
 [183.5236 ]]
160 Cost:  75.07273 
Prediction:
 [[160.06482]
 [176.3414 ]
 [181.6327 ]
 [201.20287]
 [127.94157]
 [ 98.16036]
 [160.52315]
 [133.22818]
 [165.4751 ]
 [159.65834]
 [146.39345]
 [138.72316]
 [183.73769]
 [146.84183]
 [159.28003]
 [183.72508]
 [130.74167]
 [198.29845]
 [177.09894]
 [160.92668]
 [184.53242]
 [169.93181]
 [176.33173]
 [152.49681]
 [183.54076]]
170 Cost:  74.729256 
Prediction:
 [[160.04648]
 [176.36201]
 [181.63145]
 [201.19643]
 [127.97207]
 [ 98.17997]
 [160.49875]
 [133.1815 ]
 [165.49803]
 [159.67215]
 [146.3879 ]
 [138.73456]
 [183.7424 ]
 [146.85594]
 [159.26134]
 [183.7372 ]
 [130.77455]
 [198.25607]
 [177.09767]
 [160.92   ]
 [184.51178]
 [169.94342]
 [176.30988]
 [152.49196]
 [183.55786]]
180 Cost:  74.387505 
Prediction:
 [[160.02817]
 [176.38258]
 [181.6302 ]
 [201.19   ]
 [128.0025 ]
 [ 98.19953]
 [160.47444]
 [133.13495]
 [165.52092]
 [159.68591]
 [146.38237]

In [19]:
print("Your score will be ",sess.run(hypothesis, feed_dict={X:[[100,70,101]]}))

Your score will be  [[164.95015]]


In [20]:
print("Other scores will be ",sess.run(hypothesis, feed_dict={X:[[60,70,110],[90,100,80]]}))

Other scores will be  [[213.92268]
 [174.99069]]


# 파일이 엄청 클 경우 
# 배치만큼 읽어와서 학습 시킨다.

In [22]:
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue')

# file list

In [23]:
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

In [24]:
record_defaults =[[0.],[0.],[0.],[0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

In [26]:
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)
# 10개씩 가져오라 

In [27]:
W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [28]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord = coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])#pump라고 생각하면 된다.
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

coord.request_stop()
coord.join(threads)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
0 Cost:  27153.926 
Prediction:
 [[-16.336872 ]
 [-16.775242 ]
 [-15.950918 ]
 [-11.400847 ]
 [ -4.623953 ]
 [-10.431241 ]
 [ -5.6846714]
 [ -9.159299 ]
 [ -3.1640766]
 [-10.7512455]]
10 Cost:  12.01516 
Prediction:
 [[176.56726]
 [168.87708]
 [143.04366]
 [144.28145]
 [182.81105]
 [148.96483]
 [150.95499]
 [189.10756]
 [140.79782]
 [179.42593]]
20 Cost:  12.876376 
Prediction:
 [[177.99194]
 [170.18513]
 [144.23407]
 [145.44505]
 [184.38843]
 [150.28113]
 [152.19409]
 [190.66277]
 [142.05086]
 [180.91183]]
30 Cost:  12.824826 
Prediction:
 [[177.98578]
 [170.16788]
 [144.23973]
 [145.4404 ]
 [184.40738]
 [150.30193]
 [152.19788]
 [190.66438]
 [142.07005]
 [180.92206]]
40 Cost:  12.754976 
Prediction:
 [[177.96992]
 [170.1417 ]
 [144.23734]
 [145.42786]
 [184.41568]
 [150.31386]
 [152.19327]
 [190.65546]
 [142.08081]
 [180.92224]]
50 Cost:  12.685649 
Prediction:
 [[177.95409]
 [170.11559]
 [144.23491]
 

In [29]:
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

Your score will be  [[191.54094]]


In [30]:
print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

Other scores will be  [[185.62656]
 [172.18091]]


# shuffle_batch

In [ ]:
min_after_dequeue = 10000
batch_size= 10
capacity = min_after_dequeue + 3*batch_size
example_batch, label_batch = tf.train.shuffle_batch(
    [example, label], batch_size = batch_size, capacity = capacity, 
    min_agter_deque = min_after_dequeue
)